In [141]:
# Cell 1: 필요한 라이브러리 임포트 및 설정
from PIL import Image
import random
import json
import os
import yaml

# 설정 파일 경로 (실제 경로로 변경해주세요)
config_path = "config.yaml"

# 설정 파일 로드
with open(config_path, 'r') as config_file:
    config_data = yaml.safe_load(config_file)

input_dir = config_data.get('background').get('input_dir')
output_dir = config_data.get('background').get('output_dir')
background_dir = config_data.get('background').get('background_dir')
background_size = config_data.get('background').get('size')
metadata_path = os.path.join(input_dir, "metadata.jsonl")
output_metadata_path = os.path.join(output_dir, "metadata.jsonl")

background_images = [f for f in os.listdir(background_dir)]

# 메타데이터 로드
with open(metadata_path, 'r', encoding='utf-8') as f:
    metadata_list = [json.loads(line) for line in f]

In [145]:
updated_metadata_list = []
for metadata in metadata_list:
    filename = metadata['file_name']
    input_image_path = os.path.join(input_dir, filename)
    
    if not os.path.exists(input_image_path):
        print(f"Image not found for {filename}, skipping...")
        continue
    
    # 이미지 로드
    foreground = Image.open(input_image_path).convert("RGBA")
    
    # 랜덤 배경 선택 및 로드
    background_path = os.path.join(background_dir, random.choice(background_images))
    background = Image.open(background_path).convert("RGBA")
    background = background.resize(background_size, Image.NEAREST)
    
    # 랜덤 위치 계산
    x_offset = random.randint(0, max(0, background.size[0] - foreground.size[0]))
    y_offset = random.randint(0, max(0, background.size[1] - foreground.size[1]))
    
    # 이미지 합성
    new_image = Image.new("RGBA", background.size)
    new_image.paste(background, (0, 0))
    new_image.paste(foreground, (x_offset, y_offset), foreground)
    
    # 메타데이터 업데이트
    gt_parse = json.loads(metadata['ground_truth'])['gt_parse']
    roi = gt_parse['roi']
    for key in roi:
        x, y, w, h = roi[key]
        roi[key] = [x + x_offset, y + y_offset, w, h]
    metadata['ground_truth'] = json.dumps({"gt_parse": gt_parse}, ensure_ascii=False)
    
    # 결과 저장
    output_image_path = os.path.join(output_dir, filename)
    new_image.convert("RGB").save(output_image_path)
    
    updated_metadata_list.append(metadata)
    print(f"Processed {filename}")
    

Processed note_ko_0.jpg
Processed note_ko_1.jpg
Processed note_ko_2.jpg
Processed note_ko_3.jpg
Processed note_ko_4.jpg
Processed note_ko_5.jpg
Processed note_ko_6.jpg
Processed note_ko_7.jpg
Processed note_ko_8.jpg
Processed note_ko_9.jpg
Processed note_ko_10.jpg
Processed note_ko_11.jpg
Processed note_ko_12.jpg
Processed note_ko_13.jpg
Processed note_ko_14.jpg
Processed note_ko_15.jpg
Processed note_ko_16.jpg
Processed note_ko_17.jpg
Processed note_ko_18.jpg
Processed note_ko_19.jpg
Processed note_ko_20.jpg
Processed note_ko_21.jpg
Processed note_ko_22.jpg
Processed note_ko_23.jpg
Processed note_ko_24.jpg
Processed note_ko_25.jpg
Processed note_ko_26.jpg
Processed note_ko_27.jpg
Processed note_ko_28.jpg
Processed note_ko_29.jpg
Processed note_ko_30.jpg
Processed note_ko_31.jpg
Processed note_ko_32.jpg
Processed note_ko_33.jpg
Processed note_ko_34.jpg
Processed note_ko_35.jpg
Processed note_ko_36.jpg
Processed note_ko_37.jpg
Processed note_ko_38.jpg
Processed note_ko_39.jpg
Processed 

KeyboardInterrupt: 

In [146]:
# 업데이트된 메타데이터를 JSONL 파일로 저장
with open(output_metadata_path, 'w', encoding='utf-8') as f:
    for metadata in updated_metadata_list:
        f.write(json.dumps(metadata, ensure_ascii=False) + '\n')

print("Processing complete.")

Processing complete.
